### Imports

In [30]:
import tqdm
import numpy as np
import pickle
from rich import print
import heapq
import uuid
from pprint import pprint
from collections import Counter, defaultdict
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch_dsl import Search, Index, analyzer, tokenizer
from elasticsearch_dsl.query import Q

### TF-IDF

In [31]:
from elasticsearch import Elasticsearch
client = Elasticsearch("http://localhost:9200", request_timeout=1000)

index_names = ['technical_ind', 'objective_ind']
corpuses = {'technical_ind':{}, 'objective_ind':{}}
for index_name in index_names:
    ndocs = int(client.cat.count(index=index_name, format = "json")[0]['count'])
    print(f"There are {ndocs} documents in the index '{index_name}'")


    corpus = corpuses[index_name]    # will store _normalized_ tfidf for each document, key is internal elasticsearch id, value is dictionary of term -> tf-idf weight
    for s in tqdm.tqdm(scan(client, index=index_name, query={"query" : {"match_all": {}}}), total=ndocs):
        terms = []
        freqs = []
        dfs = []

        tv = client.termvectors(index=index_name, id=s['_id'], fields=['text'], term_statistics=True, positions=False)
        if 'text' in tv['term_vectors']:   # just in case some document has no field named 'text'
            for t in tv['term_vectors']['text']['terms']:
                f = tv['term_vectors']['text']['terms'][t]['term_freq']

                terms.append(t)
                freqs.append(tv['term_vectors']['text']['terms'][t]['term_freq'])
                dfs.append(tv['term_vectors']['text']['terms'][t]['doc_freq'])

        # vector computations for tf-idf; l2-normalized for further calculations..
        tfidf = np.array(freqs) * np.log2(ndocs / np.array(dfs))
        tfidf /= np.linalg.norm(tfidf)

        # save in corpus dictionary
        corpus[s['_source']['path']] = {t: tfidf[j] for j, t in enumerate(terms)}



There are 924 documents in the index 'technical_ind'

  0%|          | 0/924 [00:00<?, ?it/s]

100%|██████████| 924/924 [00:01<00:00, 518.40it/s]


There are 924 documents in the index 'objective_ind'

100%|██████████| 924/924 [00:01<00:00, 513.67it/s]


#### Basic functions

In [32]:
def norm(d: list[tuple[str, float]]) -> float:
    return np.sqrt(sum([freq*freq for _, freq in d]))


def normalize(d1: list[tuple[str, float]]):
    normm = norm(d1)
    return [(k, v/normm) for k, v in d1]

def stemmer(query: str) -> str:
    res = ind.analyze(body={'analyzer':'default', 'text': query})
    query_stemmed = ''
    first = True
    for r in res['tokens']:
        if not first:
            query_stemmed += ' ' + r['token']
        else:
            query_stemmed += r['token']
            first = False
    return query_stemmed

### Objective_ind similarities

In [33]:
from elasticsearch.helpers import scan
from pprint import pprint
from elasticsearch import Elasticsearch
import tqdm
import numpy as np

client = Elasticsearch("http://localhost:9200", request_timeout=1000)

r = 10  # only return r top docs
queries = ['win prize many top dive trophy limit victory','learn skills dive improve gain experience', 'first try begin people knowledge start','level experiment journey collaborate experience']
sims_ob : dict[str, dict[int,float]] = {}

l2query  = [np.sqrt(len(query.split())) for query in queries]  # l2 of query assuming 0-1 vector representation

# get nr. of docs; just for the progress bar
ndocs = int(client.cat.count(index='objective_ind', format = "json")[0]['count'])

# scan through docs, compute cosine sim between query and each doc
for s in tqdm.tqdm(scan(client, index='objective_ind', query={"query" : {"match_all": {}}}), total=ndocs):
    
    docid = s['_source']['path']   # use path as id
    weights = corpuses['objective_ind'][docid]   # gets weights as a python dict of term -> weight (see remark above)
    docid = docid.split('/')[-1].replace('.txt', '')
    sims_ob[docid] = {}
    for i in range(len(queries)):
        sims_ob[docid][i] = 0.0
        for w in queries[i].split():  # gets terms as a list
            if w in weights:    # probably need to do something fancier to make sure that word is in vocabulary etc.
                sims_ob[docid][i] += weights[w]   # accumulates if w in current doc
        # normalize sim
        sims_ob[docid][i] /= l2query[i]

# now sort by cosine similarity
#sorted_answer = sorted(sims.items(), key=lambda kv: kv[1], reverse=True)

#pprint(sorted_answer[:r])


100%|██████████| 924/924 [00:00<00:00, 21411.57it/s]


#### Print de les sims

In [34]:
print('OBJECTIVE INDEX')
for i in sims_ob.keys():
    if sims_ob[i][2] > 0.1:
        print(i, sims_ob[i])

OBJECTIVE INDEX

141e6274-1c50-4244-bb4a-22e11023e6a7
{0: 0.022050362701892524, 1: 0.03815377561542171, 2: 0.11129958890849673, 3: 0.0}

65327c02-6a7b-4ff6-874a-ea0a1466b207
{0: 0.019572643415062426, 1: 0.015311247921908612, 2: 0.12114687139681465, 3: 0.0}

e2765a27-7bc1-4ac1-979e-cfc1f9c561b1
{0: 0.023807990508077755, 1: 0.034343050549433596, 2: 0.11197312302062618, 3: 0.02023049121600956}

71f07e5c-67f0-438f-8fde-098f709ca646
{0: 0.10514222715490928, 1: 0.023326527705123428, 2: 0.12700579809030244, 3: 0.0}

...

20c4e8c9-4f85-4faf-960e-be0c11d7728d
{0: 0.01346907536107509, 1: 0.023305561274489052, 2: 0.11761566971771602, 3: 0.0}

cfc1d014-515c-4d65-ada3-fee200cfd9af
{0: 0.0, 1: 0.06189783249732824, 2: 0.12588948828787758, 3: 0.0}

435aaa98-d532-4b9f-9fc2-e19781233c8b
{0: 0.04555670034129712, 1: 0.025753865469894874, 2: 0.10521961926594633, 3: 0.0}

9f283351-770f-4473-a866-0ca27e6321f1
{0: 0.0, 1: 0.013580887025102755, 2: 0.11541138257553873, 3: 0.04009777827903863}

9a19793f-5570-49c2-b99b-9af7997c5f7c
{0: 0.010133677784173949, 1: 0.006911288612944649, 2: 0.15322980079283244, 3: 0.0}

### Technical_ind similarities (no funciona, no sé per què)

In [35]:
from elasticsearch.helpers import scan
from pprint import pprint
from elasticsearch import Elasticsearch
import tqdm
import numpy as np

client = Elasticsearch("http://localhost:9200", request_timeout=1000)

r = 10  # only return r top docs
query = 'python react postgreSQL figma c++ java react pytorch sql html/css mongodb google flutter amazon raspberry tensorflow ar/vr'
sims_technical : dict[str, float] = {}

l2query  = np.sqrt(len(query.split()))  # l2 of query assuming 0-1 vector representation

# get nr. of docs; just for the progress bar
ndocs = int(client.cat.count(index='technical_ind', format = "json")[0]['count'])

# scan through docs, compute cosine sim between query and each doc
for s in tqdm.tqdm(scan(client, index='technical_ind', query={"query" : {"match_all": {}}}), total=ndocs):
    docid = s['_source']['path']   # use path as id
    weights = corpuses['technical_ind'][docid]   # gets weights as a python dict of term -> weight (see remark above)
    docid = docid.split('/')[-1].replace('.txt', '')
    sims_technical[docid] = 0.0

    for w in query.split():  # gets terms as a list
        if w in weights:    # probably need to do something fancier to make sure that word is in vocabulary etc.
                
            sims_technical[docid] += weights[w]   # accumulates if w in current doc
        # normalize sim
    sims_technical[docid] /= l2query

#sims_technical = {key: sum(subdict.values()) for key, subdict in sims_technical.items()}

  0%|          | 0/924 [00:00<?, ?it/s]

100%|██████████| 924/924 [00:00<00:00, 26537.50it/s]


In [36]:
for key, value in sims_technical.items():
    if value > 0.1:
        print(key, value)

329773c9-9ea9-45c8-84bb-d68427073632 0.10198868761551977

b1948e98-e330-4d55-aaeb-a0672ccfadc4 0.11716980838428814

8a354d53-8533-4a75-96d9-5fb93ce2019d 0.1260346529366269

c8536faf-fdc9-4617-8088-a2ec9a08c8b4 0.10041674711552839

### PROVES DIVERSES

In [37]:
from participant import load_participants
from rich import print
import uuid
import os

data_path = "data/datathon_participants.json"
participants = load_participants(data_path)

objectives : dict[uuid.UUID,str] = {}
technical : dict[uuid.UUID,str] = {}


for p in participants:
    objectives[p.id] = p.objective + " " + p.introduction
    technical[p.id] = p.technical_project + " " + p.future_excitement


word_counts : dict[str,int] = {}
for key, value in technical.items():
    for word in value.split():
        if word not in word_counts: 
            word_counts[word] = 0
        word_counts[word] += 1

sorted_answer = sorted(word_counts.items(), key=lambda kv: kv[1], reverse=True)
print(sorted_answer)
print(objectives['fcee953a-30c6-475a-b65c-ec49223281e9'])

text = 'Objectives_files/fcee953a-30c6-475a-b65c-ec49223281e9.txt'
resultat = text.split('/')[-1].replace('.txt', '')
print(resultat)

[
    ('and', 8843),
    ('to', 8598),
    ('a', 6542),
    ('the', 5464),
    ('I', 4468),
    ('that', 3023),
    ('of', 2854),
    ('was', 2360),
    ('my', 2292),
    ("I'm", 1966),
    ('on', 1942),
    ('with', 1864),
    ('in', 1518),
    ('for', 1501),
    ('project', 1368),
    ('it', 1319),
    ('It', 1095),
    ('an', 1094),
    ('about', 983),
    ('using', 952),
    ('learning', 858),
    ('me', 817),
    ('working', 762),
    ('The', 745),
    ('their', 718),
    ('where', 710),
    ('thrilled', 705),
    ('can', 692),
    ('create', 678),
    ('app', 656),
    ('next', 641),
    ('see', 623),
    ('envision', 582),
    ('machine', 581),
    ('is', 552),
    ('more', 529),
    ('this', 517),
    ('As', 503),
    ('come', 496),
    ('loved', 492),
    ('Imagine', 486),
    ('excited', 485),
    ('love', 476),
    ('but', 476),
    ('platform', 472),
    ('data', 460),
    ('make', 456),
    ('could', 451),
    ('be', 448),
    ("I'd", 448),
    ('AI-powered', 429),
    ('impact', 429),
    ('10', 408),
    ('In', 407),
    ('developing', 400),
    ("It's", 397),
    ('built', 393),
    ('personalized', 386),
    ('used', 371),
    ('skills', 370),
    ('potential', 360),
    ('technology', 359),
    ('years,', 358),
    ('by', 356),
    ('creating', 355),
    ('future', 352),
    ('really', 349),
    ('work', 349),
    ('even', 347),
    ('had', 347),
    ('so', 346),
    ('music', 341),
    ('from', 339),
    ('system', 332),
    ('-', 328),
    ('how', 320),
    ('making', 320),
    ('into', 320),
    ('part', 316),
    ('share', 315),
    ('we', 312),
    ('AI', 310),
    ('way', 307),
    ('amazing', 302),
    ('like', 295),
    ('design', 287),
    ('healthcare', 286),
    ('life', 275),
    ('revolutionize', 274),
    ('people', 272),
    ('also', 271),
    ('think', 269),
    ('contribute', 269),
    ('great', 268),
    ('users', 266),
    ('prospect', 266),
    ('passion', 264),
    ('would', 261),
    ('Seeing', 261),
    ('enjoyed', 258),
    ('health', 255),
    ('blast', 254),
    ("I've", 247),
    ('worked', 245),
    ('language', 244),
    ('social', 243),
    ('One', 242),
    ('designed', 242),
    ('been', 241),
    ('personal', 240),
    ('experience', 240),
    ('created', 237),
    ('development', 236),
    ('learn', 234),
    ('world', 233),
    ('our', 232),
    ('eager', 229),
    ('projects', 228),
    ('IoT', 225),
    ('chatbot', 224),
    ('being', 221),
    ('passionate', 221),
    ('mobile', 221),
    ('smart', 219),
    ('seeing', 218),
    ('help', 217),
    ('as', 216),
    ('or', 215),
    ('blockchain', 212),
    ('virtual', 212),
    ('This', 207),
    ('develop', 200),
    ('education', 200),
    ('innovative', 199),
    ('always', 199),
    ('super', 197),
    ('natural', 197),
    ('voice', 195),
    ('able', 194),
    ('track', 193),
    ('friends', 193),
    ("people's", 192),
    ('allowed', 191),
    ('user', 191),
    ('students', 191),
    ('vision', 189),
    ('have', 189),
    ('daily', 189),
    ('something', 187),
    ('simple', 186),
    ('out', 183),
    ('are', 182),
    ('React', 182),
    ('fascinated', 181),
    ('My', 181),
    ('got', 178),
    ('believe', 175),
    ('fun', 175),
    ('through', 174),
    ('possibilities', 172),
    ('combine', 172),
    ('provide', 171),
    ('programming', 171),
    ('medical', 170),
    ('building', 169),
    ('computer', 169),
    ('dream', 166),
    ('project,', 166),
    ('feeling', 165),
    ('lives.', 162),
    ('explore', 162),
    ('your', 161),
    ('challenge', 160),
    ('control', 159),
    ('With', 159),
    ('positive', 159),
    ('at', 157),
    ('new', 156),
    ('when', 156),
    ('processing', 154),
    ('What', 154),
    ('individuals', 153),
    ('game', 153),
    ('access', 152),
    ('financial', 150),
    ("it's", 144),
    ('tech', 144),
    ('interact', 143),
    ('idea', 142),
    ('build', 141),
    ('reality', 141),
    ('all', 141),
    ('incredible', 141),
    ('up', 141),
    ('systems', 140)

My objective for this datathon is to win, plain and simple. I've been participating in hackathons for a few years 
now, and I've come close a few times, but I'm tired of just being in the top three. I want to take home that top 
prize and prove to myself that I have what it takes to be the best. I'm ready to put in the long hours, stay 
focused, and collaborate with the right team to bring home that win. My goal is clear: I'm not here to make friends
or learn new skills, I'm here to win. Hey, I'm Sophia Estrada, an 18-year-old student with a hunger for coding 
competitions. I've been participating in hackathons for a few years now, and I'm determined to win this one. I've 
developed a passion for the tech industry, particularly IoT and blockchain. My goal is to push my skills to the 
limit and collaborate with a great team to take home the top prize. I'm confident in my abilities, and I'm ready to
put in the hard work required to succeed. Bring it on, I'm ready to rock this datathon!

fcee953a-30c6-475a-b65c-ec49223281e9

### NEW DATA

In [38]:
import json
import pathlib
import uuid
from dataclasses import dataclass
from typing import Dict, List, Literal


@dataclass
class Participant:
    id: uuid.UUID  # Unique identifier

    # Personal data
    name: str
    email: str
    age: int
    year_of_study: Literal["1st year", "2nd year", "3rd year", "4th year", "Masters", "PhD"]
    shirt_size: Literal["S", "M", "L", "XL"]
    university: str
    dietary_restrictions: Literal["None", "Vegetarian", "Vegan", "Gluten-free", "Other"]

    # Experience and programming skills
    programming_skills: Dict[str, int]
    experience_level: Literal["Beginner", "Intermediate", "Advanced"]
    hackathons_done: int

    # Interests, preferences and constraints
    interests: List[str]
    preferred_role: Literal[
        "Analysis", "Visualization", "Development", "Design", "Don't know", "Don't care"
    ]

    interest_in_challenges: List[str]
    preferred_languages: List[str]
    friend_registration: List[uuid.UUID]
    preferred_team_size: int
    availability: Dict[str, bool]

    # Description of the participant
    Tryhard: float = 0
    Learner: float = 0
    Rookie: float = 0
    Portfolio: float = 0
    Experience : float = 0

In [39]:
def load_participants(path: str, sims: dict[uuid.UUID, list[float]]) -> list[Participant]:
    if not pathlib.Path(path).exists():
        raise FileNotFoundError(
            f"The file {path} does not exist, are you sure you're using the correct path?"
        )
    if not pathlib.Path(path).suffix == ".json":
        raise ValueError(
            f"The file {path} is not a JSON file, are you sure you're using the correct file?"
        )

    participants_data = json.load(open(path))
    participants = []
    
    for participant_data in participants_data:
        # Remove fields that aren't in the Participant class
        if 'objective' in participant_data:
            del participant_data['objective']
        if 'introduction' in participant_data:
            del participant_data['introduction']
        if 'fun_fact' in participant_data:
            del participant_data['fun_fact']
        if 'future_excitement' in participant_data:
            del participant_data['future_excitement']
        if 'technical_project' in participant_data:
            del participant_data['technical_project']
            
        # Convert the ID string to UUID
        participant_id = str(participant_data['id'])
        
        # If this participant has simulation data, update their values
        if participant_id in sims.keys():
            participant_data['Tryhard'] = sims[participant_id][0]
            participant_data['Learner'] = sims[participant_id][1]
            participant_data['Rookie'] = sims[participant_id][2]
            participant_data['Portfolio'] = sims[participant_id][3]
            participant_data['Experience'] = sims_technical[participant_id]
          
        # Create the participant instance with the updated data
        participants.append(Participant(**participant_data))
    
    return participants


participants1 = load_participants('data/datathon_participants.json', sims_ob)

In [40]:
import csv
import pandas as pd

def participants_to_csv(participants: List[Participant], output_file: str):
    """Convert list of Participant objects to CSV file."""
    # Get all fields from the first participant
    fieldnames = [field for field in vars(participants[0])]
    
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write header
        writer.writerow(fieldnames)
        
        # Write each participant's data
        for participant in participants:
            row = []
            for field in fieldnames:
                value = getattr(participant, field)
                # Convert complex types to strings
                if isinstance(value, (dict, list)):
                    value = str(value)
                row.append(value)
            writer.writerow(row)

# Example usage
participants_to_csv(participants1, 'participants.csv')

In [41]:
import shutil

shutil.move('participants.csv', '/Users/cristinateixidocruilles/Desktop/Datathon24/Preprocessing/participants.csv')

'/Users/cristinateixidocruilles/Desktop/Datathon24/Preprocessing/participants.csv'